In [16]:
!pip install -U scikit-learn
!pip install hpfrec
!pip install tensorflow

import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import numpy.random as npr
import pandas as pd
import timeit

from scipy import sparse
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, LinearRegression

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: scikit-learn in /home/uk2051/.local/lib/python3.8/site-packages (1.0.2)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [17]:
countOfTopSongsToRecommend = 1000

In [18]:
def CreateExposureAndRatingsMatrixAndTopIds(maxRecoCount):
    df_ratings = pd.read_csv("/home/uk2051/ML_Project/train_0.txt", sep="\t", header=None,names=["UserId", "ItemId", "Count"])    

    
    #Unique Rating Counts
    uniqueValCounts = df_ratings["ItemId"].value_counts()
    maxSongsList =  pd.DataFrame(uniqueValCounts).head(maxRecoCount).index.tolist()
    
    
    df_exposures = df_ratings.copy()
    df_exposures['Count'] = df_exposures['Count'].where(df_exposures['Count'] == 0, 1)
    totalUsers = df_ratings["UserId"].max() + 1
    totalItems = df_ratings["ItemId"].max() + 1
    print('Total users are: ',totalUsers)
    print('Total songs are: ',totalItems)
    matrix_ratings = sparse.coo_matrix((df_ratings["Count"],(df_ratings["UserId"],df_ratings["ItemId"])),shape=(totalUsers,totalItems)).tocsr()
    matrix_exposure = sparse.coo_matrix((df_exposures["Count"],(df_exposures["UserId"],df_exposures["ItemId"])),shape=(totalUsers,totalItems)).tocsr()
    return totalUsers, totalItems, matrix_exposure, matrix_ratings, maxSongsList

In [19]:
totalUsers, totalSongs, matrix_exposure, matrix_ratings, maxSongsList = CreateExposureAndRatingsMatrixAndTopIds(countOfTopSongsToRecommend)

Total users are:  200000
Total songs are:  136736


In [20]:
#Reading thetas and betas
def GetThetaAndBeta():
    user_thetas = pd.read_csv("/home/uk2051/ML_Project/Params/Theta",header=None)
    user_betas = pd.read_csv("/home/uk2051/ML_Project/Params/Beta",header=None)
    user_betas = user_betas.T
    print("Users factors shape: ",user_thetas.shape)
    print("Songs factors shape: ",user_betas.shape)    
    return user_thetas, user_betas 

In [21]:
#Fetching theta and betas from csvs
user_thetas, item_betas = GetThetaAndBeta()

Users factors shape:  (200000, 10)
Songs factors shape:  (10, 136736)


In [22]:
def GetThetaBetaProdForSongId(user_thetas, item_betas, currSongId):
    current_song_beta = item_betas.iloc[:,currSongId]
    matVectorProd = np.matmul(user_thetas,current_song_beta)
    matVectorProd = matVectorProd.to_numpy()
    matVectorProd = np.reshape(matVectorProd,(user_thetas.shape[0],1))
    return matVectorProd

In [23]:
#print (maxSongsList)

In [24]:


reg = Ridge(normalize=True)
#reg = LinearRegression(n_jobs = 8)


print('Start Linear Regression')
tic = timeit.timeit()
#for i in range(maxSongsList):  
for i in maxSongsList:  
    a_hat_curr_song = GetThetaBetaProdForSongId(user_thetas, item_betas, i)
    curr_item_exposures = matrix_exposure[:,i].toarray()
    curr_item_ratings = matrix_ratings[:,i].toarray()
    curr_features = app_row = np.column_stack((curr_item_exposures, a_hat_curr_song))
    reg.fit(curr_features, curr_item_ratings)

toc = timeit.timeit()
print("Time Taken for training: ")
print (toc - tic, 'sec')
#    print("Iteration Success:", i)
#    print(type(curr_item_exposures))
#    print(type(curr_item_ratings))
#    print(curr_item_exposures.shape)
#    print(curr_item_ratings.shape)
#    print(curr_features.shape)



#    reg.fit(np.column_stack([a_matrix[:,i], a_hat_matrix[:,i]]), matrix[:,i])
#    allWeights[i,:] = reg.coef_
#    allIntercepts[i,:] = reg.intercept_
#    weights = reg.coef_
#    print(weights)

Start Linear Regression
Time Taken for training: 
-0.001543533056974411 sec


In [25]:
#df_test = pd.read_csv("/content/drive/MyDrive/ML_Project_Datasets/test_0.txt", sep="\t", header=None,
#                 names=["UserId", "ItemId", "Count"])
df_test1 = pd.read_csv("/home/uk2051/ML_Project/test_0.txt", sep="\t", header=None,
                 names=["UserId", "ItemId", "Count"])


In [26]:
df_test1.shape

(2000000, 3)

In [27]:
#Only predict for starting 1,0000 users
df_test = df_test1.head(10000)

In [28]:
df_test.shape

(10000, 3)

In [29]:
y_true_test = []
y_pred_test = []
for index, row in df_test.iterrows():
    curr_user = row["UserId"]
    curr_song = row["ItemId"]
    if (curr_song in maxSongsList):
        y_true = row["Count"]
        
        a_hat_curr_song = GetThetaBetaProdForSongId(user_thetas, item_betas, curr_song)
        a_hat_curr_song_curr_user = a_hat_curr_song[curr_user]
        
        curr_item_exposures = matrix_exposure[curr_user,curr_song]
        curr_item_ratings = matrix_ratings[curr_user,curr_song]

        curr_features = app_row = np.column_stack((curr_item_exposures, a_hat_curr_song_curr_user))

        
        
        y_pred = reg.predict(curr_features)        
        y_true_test.append(y_true)
        y_pred_test.append(y_pred)
#      break

In [30]:
y_pred_test = np.reshape(y_pred_test, (len(y_true_test), 1)).tolist()


#print(y_true_test)
#print(y_pred_test)

rmse_deconfounder = mean_squared_error(y_true_test, y_pred_test, squared=False)
print("Root mean squared error using deconfounder:")
print(rmse_deconfounder)

Root mean squared error using deconfounder:
3.79339894959138
